In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os
import openai


In [4]:
prompt_system = "You are a helpful assistant whose goal is to help write stories."

prompt = """continue the followin story. Write no more than 50 words. Once upon a time, in a world where animals could speak, a courageous mouse named Benjamin decided to
"""

In [5]:
response = openai.ChatCompletion.create(
model = "gpt-3.5-turbo",
messages =[
    {"role": "system", "content": prompt_system},
    {"role":"user", "content":prompt}
])

In [6]:
response.choices[0]['message']['content']

'embark on a journey to save his village from a menacing hawk. Armed with quick wits and a heart full of determination, Benjamin set out into the unknown.'

answer changes even with a single spelling mistake

In [14]:

prompt_system = "You are a helpful assistant whose goal is to help write stories."

# here i have given courageous as courageious
prompt = """continue the followin story. Write no more than 50 words. Once upon a time, in a world where animals could speak, a courageious mouse named Benjamin decided to
"""

response = openai.ChatCompletion.create(
model = "gpt-3.5-turbo",
messages =[
    {"role": "system", "content": prompt_system},
    {"role":"user", "content":prompt}
])

response.choices[0]['message']['content']

'travel to the farthest reaches of the forest to seek out the legendary wise owl. With determination in his heart, Benjamin embarked on his journey, unaware of the challenges that awaited him.'

### Product Description

In [18]:
prompt_system = "You are a helpful assistant whose goal is to help write a product description."

# here i have given courageous as courageious
prompt = """write a captivating product description for a luxurious, handcrafted, limited-edition fountain pen made from rosewood and gold. Write no more than 50 words"""

response = openai.ChatCompletion.create(
model = "gpt-3.5-turbo",
messages =[
    {"role": "system", "content": prompt_system},
    {"role":"user", "content":prompt}
])

response.choices[0]['message']['content']

'Indulge in the ultimate writing experience with our exquisite limited-edition fountain pen. Handcrafted from lustrous rosewood and accented with luxurious gold, this pen exudes elegance and sophistication. Elevate your writing to new heights with this unique masterpiece.'

how does llm understand the word limit and generate only so much?<br>
how does it keep track of it's word generation?

<b> Language models are the offspring of the sum of all human linguistic output </b>
- https://generative.ink/posts/methods-of-prompt-programming/


## Prompting Techniques

### Zero shot prompting

In [24]:
prompt_system = "You are a helpful assistant whose goal is to write short poems."

prompt = """Write a short poem about {topic}."""

response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
messages = [
    {
        "role": "system", "content": prompt_system,
        
    },
    {
        "role": "system", "content": prompt.format(topic="summer")
    }
])

In [26]:
print(response.choices[0]['message']['content'])

Golden sunshine fills the skies,
Warm breezes kiss my skin,
Days stretch out with no goodbyes,
In summer's sweet embrace we spin.


### 2. In-Context Learning And Few-Shot Prompting

In [30]:
prompt_system = "You are a helpful assistant whose goal is to write short poems."

prompt = """Write a short poem about {topic}"""

examples = {
    "nature": """Birdsong fills the air, \nMountains high and valleys deep,\nNature's music sweet""",
    "winter": """Snow blankets the ground,\n Silence is the only sound, \nWinter's beauy found."""
}

response = openai.ChatCompletion.create(
model = "gpt-3.5-turbo",
messages = [
    { "role": "system", "content": prompt_system},
    {"role": "user", "content": prompt.format(topic="nature")},
    {"role": "assistant", "content": examples["nature"]},
    {"role": "user", "content": prompt.format(topic="winter")},
    {"role": "assistant", "content": examples["winter"]},
    {"role": "user", "content": prompt.format(topic="winter")},
])
print(response.choices[0]['message']['content'])

Icy breath in the air,
Frost dusts the world with care,
Winter's chill, nature's flair.


#### Few-Shot Prompting
using langchain framework

In [8]:
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

In [14]:
llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature=0)

examples = [
    {"color":"red", "emotion":"passion"},
    {"color":"blue", "emotion":"serenity"},
    {"color":"green", "emotion":"tranquility"}
]


example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""

example_prompt = PromptTemplate(input_variables=["color", "emotion"],
                               template=example_formatter_template,)


few_shot_prompt = FewShotPromptTemplate(
examples = examples,
example_prompt=example_prompt,
prefix = """Here are some examples of colors and the emotions associated with them:\n\n""",
suffix="""\n\nNow, given a new color, identify the emotion associated with it:\n\nColor:{input}\nEmotion:""",
input_variables=["input"],
example_separator="\n")

In [21]:
formatted_prompt = few_shot_prompt.format(input="brown")

# Creating thee LLMChain for the prompt


chain = LLMChain(llm=llm, prompt=PromptTemplate(template=formatted_prompt, input_variables=[]))


# Running LLM chain 

response = chain.run({})

print("Color: brown")
print("Emotion: ", response)

Color: brown
Emotion:  earthiness


### Role Prompting
Defining role in prompt , helping the llm to assume a specific role. helps it to tailor the outputs w.r.t those roles. shows how training on large data it can create its own alter-ego.
1. papers regarding role prompting?
2. role of temperature and how it works behind the scene?

In [27]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


template = """
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about {theme} in the year {year}?
"""

prompt= PromptTemplate(input_variables=["theme", "year"], 
                      template=template
                      )



input_data = {"theme": "Interstellar travel", "year": "3030"}


# Creating LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

response = chain.run(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print("Ai generated song title: ", response)

Theme: interstellar travel
Year: 3030
Ai generated song title:  "Galactic Odyssey: Journey to the Stars"


What makes a good prompt?
1. Precise Direction - making sure it gets what need to be genrated
2. specificity - providing proper specific instruction like a conductor in a band
3. Promoting Creativity - not giving specific format or style helping for creativity 
4. Concentrated on the task - concentrates exclusive on the task 

### Chain Prompting

In [38]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

#Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer = """
prompt_question =PromptTemplate(template=template_question, input_variables=[])


#Prompt 2 
template_fact = """Provide a brief description of {scientist}'s theory of general relativity.
Answer: """

prompt_fact = PromptTemplate(template=template_fact, input_variables=['scientist'])



# creating the chain for first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# Run the LLM of 1st prompt
response_question = chain_question.run({})


# extracting scienstist name
scientist = response_question.strip()

chain_answer = LLMChain(llm=llm, prompt=prompt_fact)

input_data = { "scientist" : scientist }
response_fact = chain_answer.run(input_data)

print("Scientist", scientist)
print("Fact:", response_fact)

Scientist Albert Einstein
Fact: Albert Einstein's theory of general relativity is a theory of gravitation that describes the force of gravity as a curvature of spacetime caused by mass and energy. According to this theory, massive objects like planets and stars warp the fabric of spacetime, causing other objects to move along curved paths. General relativity also predicts the existence of phenomena such as black holes, gravitational waves, and the bending of light around massive objects.


### Chain of Thought Prompting
Method designed to prompt LLM to articulate their thoughts process, to enhance the results. It's like mentoring and helping a junior to reach results by directing their thought process with reasoning.

<i>Advantages of using Cot in langchain.</i>
1. simplifies complex tasks by enabling to breakdown the challenging tasks which is valuable for tasks requiring calculations, logical analysis, or multi-step reasoning.
2. it can guide the model through a series of related prompts, fostering more coherent and contextually appropriate outputs.

<i> Limitations</i>
1. helpful with large model i.e. 100 Billion parameters and above
2. It's effectiveness can vary across different types of tasks.

#### Tips for effective Prompt Engineering
* Be specific
* Force conciseness
* Encourage the model to describe why it is the way it is

Examples of good prompt Engineering

In [39]:
from langchain import FewShotPromptTemplate, PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

examples = [
    {
        "query": "what's the secret to the happiness?",
        "answer": """Finding balance in life and learning to enjoy the small moments."""
    },{
        "query": "How can i become more productive?",
        "answer":"""Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."""
    }
]

example_template = """
User: {query}
Ai : {answer}
"""

example_prompt = PromptTemplate(
input_variables = ["query", "answer"],
template=example_template
)


prefix = """The following are excerpts from the conversations with an AI life coach. 
The assistant provides insightful and practical advice to the user's questions. Here are some examples:
"""

# what will happen if i add a new lines after colon . what kind of response can be generated?
suffix = """
User : {query},
AI :"""


few_shot_prompt_template = FewShotPromptTemplate(
examples = examples,
example_prompt=example_prompt,
prefix = prefix,
suffix=suffix,
input_variables=["query"],
example_separator="\n\n")


chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)


user_query = "What are some tips for improving communication skills?"

response = chain.run({"query": user_query})

print("User Query:", user_query)
print("AI Response:", response)

User Query: What are some tips for improving communication skills?
AI Response: Practice active listening, be clear and concise in your communication, and work on empathizing with others to understand their perspective better.


1. what is 696x87?
2. what is 696x87? 
<br>Let's think step by step encouraging the model to explain its reasoning leads to more accurate results.
this doesn't give always correct answers so giving some context before (few shot prompting) helps model imitate the methodology to produce answers

here in this example
* sets a clear context in the prefix
* Utilizes Examples
* Distinguishes Between Examples and Actual Query
* Includes a Clear Suffix for User Input and AI Response

## Zero shot Prompting
1. direct task specifications 
2. proxy task specification


